# Reading Weather Data

After we compiled and dumped the bikes data into a CSV, now we join with weather data.

The weather source is [darksky.net](https://darksky.net/40.7062,-73.6187)

In [1]:
import pandas as pd
import json

test_weather_file = 'data/weather/1476871201_weather.json'

def load_weather_to_json(fname):
    with open(fname, 'r') as f:
        return json.loads(f.read())['currently']
    
load_weather_to_json(test_weather_file)

{'apparentTemperature': 66.94,
 'cloudCover': 0.61,
 'dewPoint': 62.08,
 'humidity': 0.84,
 'icon': 'partly-cloudy-night',
 'nearestStormBearing': 217,
 'nearestStormDistance': 50,
 'ozone': 274.7,
 'precipIntensity': 0,
 'precipProbability': 0,
 'pressure': 1011.82,
 'summary': 'Mostly Cloudy',
 'temperature': 66.94,
 'time': 1476871201,
 'visibility': 9.6,
 'windBearing': 242,
 'windSpeed': 4.64}

In [4]:
# Load all weather data.
from os import listdir
from os.path import isfile, join

weather_path = 'data/weather/'
weather_files = sorted([f for f in listdir(weather_path) if isfile(join(weather_path, f))])

print('# files:', len(weather_files))
weather_files[:10]

# files: 2364


['1476711733_weather.json',
 '1476712801_weather.json',
 '1476713701_weather.json',
 '1476714601_weather.json',
 '1476715502_weather.json',
 '1476716401_weather.json',
 '1476717302_weather.json',
 '1476718202_weather.json',
 '1476719101_weather.json',
 '1476720001_weather.json']

In [5]:
all_data = []
for fname in weather_files:
    record = load_weather_to_json(weather_path + fname)
    all_data.append(record)

In [6]:
data_frame = pd.DataFrame(all_data)
data_frame['ts'] = data_frame['time']
data_frame['time'] = pd.to_datetime(data_frame['time'], unit='s')
data_frame.sort_values('time', inplace=True)
data_frame = data_frame.drop_duplicates(subset=['time'])

In [7]:
data_frame[:5]

,apparentTemperature,cloudCover,dewPoint,humidity,icon,nearestStormBearing,nearestStormDistance,ozone,precipIntensity,precipIntensityError,precipProbability,precipType,pressure,summary,temperature,time,visibility,windBearing,windSpeed,ts
0,66.57,0.55,60.79,0.82,partly-cloudy-day,326.0,102,283.35,0.0,NaN,0.0,NaN,1014.94,Partly Cloudy,66.57,2016-10-17 13:42:13,8.76,241,4.22,1476711733
1,66.34,0.12,59.97,0.80,clear-day,331.0,97,283.43,0.0,NaN,0.0,NaN,1014.89,Clear,66.34,2016-10-17 14:00:01,8.80,251,2.90,1476712801
2,67.74,0.16,60.53,0.78,clear-day,335.0,98,283.42,0.0,NaN,0.0,NaN,1014.82,Clear,67.74,2016-10-17 14:15:01,8.98,247,3.15,1476713701
3,69.09,0.08,61.17,0.76,clear-day,336.0,99,283.40,0.0,NaN,0.0,NaN,1014.61,Clear,69.09,2016-10-17 14:30:01,8.39,244,2.67,1476714601
4,69.99,0.11,61.69,0.75,clear-day,341.0,96,283.39,0.0,NaN,0.0,NaN,1014.60,Clear,69.99,2016-10-17 14:45:02,8.59,242,2.91,1476715502


From the data we can see a couple of rare fields:
* `precipIntensityError`: Intensity measurement error, only when raining
* `precipType`: Only filled during rain

In [8]:
print(data_frame['precipIntensityError'].unique())
print(data_frame['precipType'].unique())

[    nan  0.0004  0.0011  0.0009  0.0008  0.0007  0.0052  0.0012  0.0023
  0.0002  0.0003  0.0021  0.0014  0.002   0.0005  0.0001  0.0029  0.0018
  0.0182  0.0186  0.0016  0.001   0.0006  0.0013  0.0022  0.0033  0.0019
  0.003   0.0047  0.0026  0.0042  0.0034  0.0015  0.0025  0.0041  0.0055
  0.0202  0.0141  0.0024  0.      0.0017]
[nan 'rain']


In [9]:
start_ts = int(min(data_frame['time']).timestamp())
end_ts = int(max(data_frame['time']).timestamp())
print(start_ts, end_ts)

1476726133 1478858402


In [10]:
# Save this data frame to a file
data_frame.to_csv('data/weather_{}_{}.csv'.format(start_ts, end_ts), index=False)

# Joining to Bikes Data

The problem here is that the bikes update timestamps don't match nicely with the weather timestamp. For each bike record, we need to find the weather record that most closely matches. There are some alternatives:
* Pick the nearest record
* Pick the nearest future record
* Pick the nearest past record

In [11]:
import numpy as np

time_index = pd.Index(data_frame['time'])
print(time_index.is_monotonic_increasing)
print(time_index.get_loc(1476828133, method='nearest'))
times = np.array(data_frame['ts'])
times[:10]

True
0


array([1476711733, 1476712801, 1476713701, 1476714601, 1476715502,
       1476716401, 1476717302, 1476718202, 1476719101, 1476720001])

In [21]:
# Load the bikes csv
bikes_file = 'data/bikes_1476723624_1478858697.csv'
bikes = pd.read_csv(bikes_file)
print('# bikes:', len(bikes))
bikes[:3]

# bikes: 1311567


,eightd_has_available_keys,is_installed,is_renting,is_returning,last_reported,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,station_id
0,False,1,1,1,2016-10-17 13:00:24,12,0,13,0,3377
1,False,1,1,1,2016-10-17 13:00:45,3,0,24,0,372
2,False,1,1,1,2016-10-17 13:01:41,0,1,23,0,262


In [24]:
bikes['last_reported'] = pd.to_datetime(bikes['last_reported'], infer_datetime_format=True)
bikes['last_reported'][:10]
bikes = bikes[bikes['last_reported'] > pd.to_datetime('2000-01-01 00:00', infer_datetime_format=True)]
print('# bikes:', len(bikes))
bikes[:3]

# bikes: 1311519


,eightd_has_available_keys,is_installed,is_renting,is_returning,last_reported,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,station_id
0,False,1,1,1,2016-10-17 13:00:24,12,0,13,0,3377
1,False,1,1,1,2016-10-17 13:00:45,3,0,24,0,372
2,False,1,1,1,2016-10-17 13:01:41,0,1,23,0,262


In [25]:
def to_time(ts):
    if isinstance(ts, type(pd.NaT)) or ts is None or isinstance(ts, float):
        return 0
    return int(ts.timestamp())

bike_times = [to_time(bt) for bt in bikes['last_reported']]
bike_times[10101:10111]

[1476738517,
 1476738518,
 1476738519,
 1476738519,
 1476738521,
 1476738521,
 1476738523,
 1476738525,
 1476738525,
 1476738525]

In [26]:
# Now, find the closest weather timestamps in the bike data
print(len(bike_times))
print(len(times))
weather_times = np.searchsorted(times, bike_times) - 1
weather_ts = times[weather_times]
diff = bike_times - weather_ts
print('Min diff:', min(diff))
print('Max diff:', max(diff))

print(times[:14])
print(len(weather_times))
weather_times[-50:]
weather_ts[-50:]

1311519
2364
Min diff: 1
Max diff: 18295
[1476711733 1476712801 1476713701 1476714601 1476715502 1476716401
 1476717302 1476718202 1476719101 1476720001 1476720902 1476721801
 1476722702 1476723601]
1311519


array([1478840402, 1478840402, 1478840402, 1478840402, 1478840402,
       1478840402, 1478840402, 1478840402, 1478840402, 1478840402,
       1478840402, 1478840402, 1478840402, 1478840402, 1478840402,
       1478840402, 1478840402, 1478840402, 1478840402, 1478840402,
       1478840402, 1478840402, 1478840402, 1478840402, 1478840402,
       1478840402, 1478840402, 1478840402, 1478840402, 1478840402,
       1478840402, 1478840402, 1478840402, 1478840402, 1478840402,
       1478840402, 1478840402, 1478840402, 1478840402, 1478840402,
       1478840402, 1478840402, 1478840402, 1478840402, 1478840402,
       1478840402, 1478840402, 1478840402, 1478840402, 1478840402])

In [27]:
# bike_times of 0 will turn into weather timestamp 0
wt_for_bikes = times[weather_times] * np.minimum(bike_times, 1)
print(len(wt_for_bikes))
print(wt_for_bikes[:30])
print(wt_for_bikes[10101:10131])
print(wt_for_bikes[-30:])

1311519
[1476723601 1476723601 1476723601 1476723601 1476723601 1476723601
 1476723601 1476723601 1476723601 1476723601 1476723601 1476723601
 1476723601 1476723601 1476723601 1476723601 1476723601 1476723601
 1476723601 1476723601 1476723601 1476723601 1476723601 1476723601
 1476723601 1476723601 1476723601 1476723601 1476723601 1476723601]
[1476738001 1476738001 1476738001 1476738001 1476738001 1476738001
 1476738001 1476738001 1476738001 1476738001 1476738001 1476738001
 1476738001 1476738001 1476738001 1476738001 1476738001 1476738001
 1476738001 1476738001 1476738001 1476738001 1476738001 1476738001
 1476738001 1476738001 1476738001 1476738001 1476738001 1476738001]
[1478840402 1478840402 1478840402 1478840402 1478840402 1478840402
 1478840402 1478840402 1478840402 1478840402 1478840402 1478840402
 1478840402 1478840402 1478840402 1478840402 1478840402 1478840402
 1478840402 1478840402 1478840402 1478840402 1478840402 1478840402
 1478840402 1478840402 1478840402 1478840402 1478840

In [28]:
bikes['weather_ts'] = wt_for_bikes
bikes[10101:10111]

,eightd_has_available_keys,is_installed,is_renting,is_returning,last_reported,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,station_id,weather_ts
10101,False,1,1,1,2016-10-17 17:08:37,3,0,52,0,359,1476738001
10102,False,1,1,1,2016-10-17 17:08:38,1,0,22,0,302,1476738001
10103,False,1,1,1,2016-10-17 17:08:39,19,4,7,0,3183,1476738001
10104,True,1,1,1,2016-10-17 17:08:39,1,3,55,0,3295,1476738001
10105,False,1,1,1,2016-10-17 17:08:41,1,0,35,0,2012,1476738001
10106,False,1,1,1,2016-10-17 17:08:41,13,1,17,0,3090,1476738001
10107,False,1,1,1,2016-10-17 17:08:43,1,0,24,0,3303,1476738001
10108,False,1,1,1,2016-10-17 17:08:45,1,0,34,0,3294,1476738001
10109,False,1,1,1,2016-10-17 17:08:45,0,0,31,0,428,1476738001
10110,False,1,1,1,2016-10-17 17:08:45,4,0,26,0,498,1476738001


In [29]:
# Now do the join.
bikes_and_weather = pd.merge(bikes, data_frame, left_on='weather_ts', right_on='ts').drop(['ts'], axis=1)

In [30]:
bikes_and_weather[20130:20150]

,eightd_has_available_keys,is_installed,is_renting,is_returning,last_reported,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,station_id,...,precipIntensityError,precipProbability,precipType,pressure,summary,temperature,time,visibility,windBearing,windSpeed
20130,False,1,1,1,2016-10-17 20:16:28,0,0,19,0,3042,...,NaN,0.0,NaN,1012.22,Clear,73.37,2016-10-18 00:15:01,9.83,235,4.76
20131,False,1,1,1,2016-10-17 20:16:28,2,0,23,0,396,...,NaN,0.0,NaN,1012.22,Clear,73.37,2016-10-18 00:15:01,9.83,235,4.76
20132,False,1,1,1,2016-10-17 20:16:29,41,0,1,0,348,...,NaN,0.0,NaN,1012.22,Clear,73.37,2016-10-18 00:15:01,9.83,235,4.76
20133,False,1,1,1,2016-10-17 20:16:30,13,0,29,0,401,...,NaN,0.0,NaN,1012.22,Clear,73.37,2016-10-18 00:15:01,9.83,235,4.76
20134,False,1,1,1,2016-10-17 20:16:31,19,1,8,1,503,...,NaN,0.0,NaN,1012.22,Clear,73.37,2016-10-18 00:15:01,9.83,235,4.76
20135,False,1,1,1,2016-10-17 20:16:32,4,0,26,0,498,...,NaN,0.0,NaN,1012.22,Clear,73.37,2016-10-18 00:15:01,9.83,235,4.76
20136,False,1,1,1,2016-10-17 20:16:33,45,0,0,0,195,...,NaN,0.0,NaN,1012.22,Clear,73.37,2016-10-18 00:15:01,9.83,235,4.76
20137,False,1,1,1,2016-10-17 20:16:33,12,0,23,0,441,...,NaN,0.0,NaN,1012.22,Clear,73.37,2016-10-18 00:15:01,9.83,235,4.76
20138,False,1,1,1,2016-10-17 20:16:34,19,0,8,0,353,...,NaN,0.0,NaN,1012.22,Clear,73.37,2016-10-18 00:15:01,9.83,235,4.76
20139,False,1,1,1,2016-10-17 20:16:35,6,0,25,0,365,...,NaN,0.0,NaN,1012.22,Clear,73.37,2016-10-18 00:15:01,9.83,235,4.76


In [31]:
# Save results to file
bike_start_ts = int(min(bikes_and_weather['last_reported']).timestamp())
bike_end_ts = int(max(bikes_and_weather['last_reported']).timestamp())
print(bike_start_ts, bike_end_ts)

1476723624 1478858697


In [32]:
bikes_and_weather.to_csv('data/bikes_and_weather_{}_{}.csv'.format(bike_start_ts, bike_end_ts), index=False)

In [33]:
len(bikes_and_weather)

1311519